# Interpretability examples

This notebook demonstrates interpretability by decomposing vectors.

## Perform training and inference

In [ ]:
import pandas as pd
import numpy as np

import torchhd as hd
import torch

import gc

from src.vsa_encoding import *

temporal = True
d = 10000
bins = 3
clusters = 10


for i in range(1):
    
    Joints = hd.random(5,d)
    Speeds = hd.random(bins,d)
    Aspect = hd.random(bins,d)
    Pos = hd.random(bins*bins+1,d)
    Times = hd.random(bins,d)
    Wrists = hd.random(5,d)
    Postures = hd.random(5, d)
    Features = hd.random(6,d)

    gc.collect()

    inference_components = {}

    df = pd.read_csv('./tracked_poses_s.csv')

    detections, aspect_thresh, time_thresh = build_encoding_df(df, bins=bins, aspect_thresh=None, time_thresh=None)

    vsa = encode_vsa(detections, Features, Pos, Aspect, Times, Postures, bins=bins)

    if temporal:
        temporal_vecs = build_temporal_enc(detections, vsa)
        temporal_vecs = torch.stack(temporal_vecs)
    else:
        temporal_vecs = torch.stack(vsa)

    prototypes, kmeans = cluster(temporal_vecs, clusters)

    if kmeans is None:
        labels = np.array([0] * len(temporal_vecs))
    else:
        labels = kmeans.labels_

    print(prototypes.shape, type(labels))

    thresholds = compute_cluster_thresholds(prototypes, temporal_vecs, labels)
    fine_tuned_prototypes = fine_tune_prototypes(prototypes, temporal_vecs, labels)
    updated_thresholds = compute_cluster_thresholds(fine_tuned_prototypes, temporal_vecs, labels)

    print('Prototypes finetuned')

    test = pd.read_csv('./test_tracked_poses_m.csv')
    test_detections, _,  _ = build_encoding_df(test, aspect_thresh=aspect_thresh, time_thresh=time_thresh)
    test_vsa = encode_vsa(test_detections, Features, Pos, Aspect, Times, Postures, bins=bins)
    if temporal:
        test_temporal_vecs = build_temporal_enc(test_detections, test_vsa)
        test_temporal_vecs = torch.stack(test_temporal_vecs)

    else:
        test_temporal_vecs = torch.stack(test_vsa)

    results = evaluate_test_vectors(fine_tuned_prototypes, updated_thresholds, test_temporal_vecs)

    results = pd.DataFrame({
        'video': test_detections['video'],
        'personID': test_detections['personID'],
        'frameID': test_detections['frameID'],
        'AnomalyScore': results[0],
        'AnomalyLabel': results[1],
        'AnomalyThreshold': results[2],
        'RandomSeed': seed,
        'Iteration': i
    })

/Users/joelewis/Desktop/Dissertation/NeuroSymbol-CUHK/vsa_encoding.py:212: RuntimeWarning: invalid value encountered in divide
  cos_theta = np.sum(ba * bc, axis=1) / (np.linalg.norm(ba, axis=1) * np.linalg.norm(bc, axis=1))
/Users/joelewis/Desktop/Dissertation/NeuroSymbol-CUHK/vsa_encoding.py:244: RuntimeWarning: All-NaN axis encountered
  knee_angle = np.nanmax([l_angle, r_angle], axis=0)
100%|██████████| 89313/89313 [00:05<00:00, 15445.20it/s]
89313it [00:08, 10640.38it/s]


torch.Size([10, 10000]) <class 'numpy.ndarray'>
Prototypes finetuned


/Users/joelewis/Desktop/Dissertation/NeuroSymbol-CUHK/vsa_encoding.py:212: RuntimeWarning: invalid value encountered in divide
  cos_theta = np.sum(ba * bc, axis=1) / (np.linalg.norm(ba, axis=1) * np.linalg.norm(bc, axis=1))
/Users/joelewis/Desktop/Dissertation/NeuroSymbol-CUHK/vsa_encoding.py:244: RuntimeWarning: All-NaN axis encountered
  knee_angle = np.nanmax([l_angle, r_angle], axis=0)
100%|██████████| 85747/85747 [00:05<00:00, 14611.94it/s]
85747it [00:07, 10846.61it/s]


## Example One: Video 21 - Child running near camera

In [186]:
interp_test = results

In [187]:
interp_test['video'] = interp_test.apply(lambda row: int(row['video'].split('_')[0]), axis=1)

In [188]:
interp_test['VecIndex'] = interp_test.index

In [189]:
at_t_00 = interp_test[(interp_test['frameID'] == 0) & (interp_test['personID'] == 1913) & (interp_test['video'] == 21)]
at_t_20 = interp_test[(interp_test['frameID'] ==  20) & (interp_test['personID'] == 1913) & (interp_test['video'] == 21)]
at_t_40 = interp_test[(interp_test['frameID'] ==  40) & (interp_test['personID'] == 1913) & (interp_test['video'] == 21)]

In [190]:
vec_at_00 = test_temporal_vecs[at_t_00.index]
vec_at_20 = test_temporal_vecs[at_t_20['VecIndex'].dropna().to_numpy()]
vec_at_40 = test_temporal_vecs[at_t_40['VecIndex'].dropna().to_numpy()]

In [191]:
closest_prototype_00 = fine_tuned_prototypes[hd.cosine_similarity(vec_at_00, fine_tuned_prototypes).argmax(dim=1)]
closest_prototype_20 = fine_tuned_prototypes[hd.cosine_similarity(vec_at_20, fine_tuned_prototypes).argmax(dim=1)]
closest_prototype_40 = fine_tuned_prototypes[hd.cosine_similarity(vec_at_40, fine_tuned_prototypes).argmax(dim=1)]
closest_prototype_00, closest_prototype_20, closest_prototype_40

(MAPTensor([[-1., -1., -1.,  ...,  1.,  1.,  1.]]),
 MAPTensor([[ 1., -1., -1.,  ..., -1.,  1., -1.]]),
 MAPTensor([[-1., -1., -1.,  ...,  1.,  1.,  1.]]))

In [192]:
# Describe prototypes that are closest
aspect_labels = ['far', 'middle', 'close']
time_labels = ['short', 'normal', 'long']
posture_labels = ['crouching', 'standing', 'leaning', 'unknown', 'sitting']

def interpret_vec(vec):
    aspect = aspect_labels[np.argmax(hd.cosine_similarity(hd.bind(Features[2], vec), Aspect))]
    posture = posture_labels[np.argmax(hd.cosine_similarity(hd.bind(Features[3], vec), Postures))]
    time = time_labels[np.argmax(hd.cosine_similarity(hd.bind(Features[0], vec), Times))]
    position = np.argmax(hd.cosine_similarity(hd.bind(Features[1], vec), Pos)).item()
    
    print(f"Vector: Aspect: {aspect}, Time: {time}, Position:{position}, Posture: {posture}")


In [193]:
interp_test[(interp_test['video'] == 21) & (interp_test['personID'] == 1913) & (interp_test['frameID'] == 0)]

,video,personID,frameID,AnomalyScore,AnomalyLabel,AnomalyThreshold,RandomSeed,Iteration,VecIndex
85231,21,1913,0,0.499768,MAPTensor(True),0.606167,2993,0,85231


In [194]:
interpret_vec(vec_at_00)

Vector: Aspect: close, Time: short, Position:6, Posture: standing


In [195]:
interpret_vec(closest_prototype_00)

Vector: Aspect: middle, Time: short, Position:5, Posture: standing


In [196]:
results[(results['video'] == 21) & (results['frameID'] ==20) & (results['personID'] == 1913)]

,video,personID,frameID,AnomalyScore,AnomalyLabel,AnomalyThreshold,RandomSeed,Iteration,VecIndex
85251,21,1913,20,0.315072,MAPTensor(True),0.714812,2993,0,85251


In [197]:
interpret_vec(vec_at_20)

Vector: Aspect: close, Time: short, Position:5, Posture: unknown


In [198]:
interpret_vec(closest_prototype_20)

Vector: Aspect: middle, Time: short, Position:1, Posture: unknown


In [199]:
results[(results['video'] == 21) & (results['frameID'] ==40) & (results['personID'] == 1913)]

,video,personID,frameID,AnomalyScore,AnomalyLabel,AnomalyThreshold,RandomSeed,Iteration,VecIndex
85271,21,1913,40,0.26071,MAPTensor(True),0.547292,2993,0,85271


In [200]:
interpret_vec(vec_at_40)

Vector: Aspect: close, Time: normal, Position:5, Posture: leaning


In [201]:
interpret_vec(closest_prototype_40)

Vector: Aspect: middle, Time: normal, Position:2, Posture: standing


In [ ]:
interpret_vec(hd.permute(vec_at_40, shifts=-2))

Vector: Aspect: close, Time: long, Position:8, Posture: unknown


: 

## Example 2: Man running across scene

Described at frame level.

In [202]:
at_t_580 = interp_test[(interp_test['frameID'] == 580) & (interp_test['personID'] == 1145) & (interp_test['video'] == 3)]
at_t_610 = interp_test[(interp_test['frameID'] == 610) & (interp_test['personID'] == 1162) & (interp_test['video'] == 3)]
at_t_640 = interp_test[(interp_test['frameID'] == 640) & (interp_test['personID'] == 1165) & (interp_test['video'] == 3)]


In [203]:
vec_at_580 = test_temporal_vecs[at_t_580.index]
vec_at_610 = test_temporal_vecs[at_t_610.index]
vec_at_640 = test_temporal_vecs[at_t_640.index]

In [204]:
closest_prototype_580 = fine_tuned_prototypes[hd.cosine_similarity(vec_at_580, fine_tuned_prototypes).argmax(dim=1)]
closest_prototype_610 = fine_tuned_prototypes[hd.cosine_similarity(vec_at_610, fine_tuned_prototypes).argmax(dim=1)]
closest_prototype_640 = fine_tuned_prototypes[hd.cosine_similarity(vec_at_640, fine_tuned_prototypes).argmax(dim=1)]
closest_prototype_580, closest_prototype_610, closest_prototype_640

(MAPTensor([[1., 1., 1.,  ..., 1., 1., 1.]]),
 MAPTensor([[ 1., -1., -1.,  ...,  1.,  1.,  1.]]),
 MAPTensor([[-1., -1., -1.,  ...,  1.,  1.,  1.]]))

In [205]:
at_t_580

,video,personID,frameID,AnomalyScore,AnomalyLabel,AnomalyThreshold,RandomSeed,Iteration,VecIndex
18485,3,1145,580,0.712279,MAPTensor(False),0.499652,2993,0,18485


In [206]:
interpret_vec(vec_at_580)

Vector: Aspect: middle, Time: long, Position:2, Posture: standing


In [207]:
interpret_vec(closest_prototype_580)

Vector: Aspect: middle, Time: long, Position:2, Posture: standing


In [208]:
at_t_610

,video,personID,frameID,AnomalyScore,AnomalyLabel,AnomalyThreshold,RandomSeed,Iteration,VecIndex
19058,3,1162,610,0.300814,MAPTensor(True),0.55548,2993,0,19058


In [209]:
interpret_vec(vec_at_610)

Vector: Aspect: close, Time: short, Position:5, Posture: unknown


In [210]:
interpret_vec(closest_prototype_610)

Vector: Aspect: middle, Time: short, Position:2, Posture: standing


In [211]:
at_t_640

,video,personID,frameID,AnomalyScore,AnomalyLabel,AnomalyThreshold,RandomSeed,Iteration,VecIndex
19364,3,1165,640,0.816047,MAPTensor(False),0.583249,2993,0,19364


In [212]:
interpret_vec(vec_at_640)

Vector: Aspect: middle, Time: normal, Position:1, Posture: standing


In [213]:
interpret_vec(closest_prototype_640)

Vector: Aspect: middle, Time: normal, Position:1, Posture: standing


## Example 3: Man walking wrong way

Highlights system sensitivity to aspect feature and ability to diagnose problems.

In [214]:
at_t_100 = interp_test[(interp_test['frameID'] == 100) & (interp_test['personID'] == 435) & (interp_test['video'] == 6)] 
at_t_200 = interp_test[(interp_test['frameID'] == 200) & (interp_test['personID'] == 435) & (interp_test['video'] == 6)] 
at_t_300 = interp_test[(interp_test['frameID'] == 300) & (interp_test['personID'] == 435) & (interp_test['video'] == 6)] 
at_t_400 = interp_test[(interp_test['frameID'] == 400) & (interp_test['personID'] == 435) & (interp_test['video'] == 6)] 


In [215]:
vec_at_100 = test_temporal_vecs[at_t_100.index]
vec_at_200 = test_temporal_vecs[at_t_200.index]
vec_at_300 = test_temporal_vecs[at_t_300.index]
vec_at_400 = test_temporal_vecs[at_t_400.index]

In [216]:
closest_prototype_100 = fine_tuned_prototypes[hd.cosine_similarity(vec_at_100, fine_tuned_prototypes).argmax(dim=1)]
closest_prototype_200 = fine_tuned_prototypes[hd.cosine_similarity(vec_at_200, fine_tuned_prototypes).argmax(dim=1)]
closest_prototype_300 = fine_tuned_prototypes[hd.cosine_similarity(vec_at_300, fine_tuned_prototypes).argmax(dim=1)]
closest_prototype_400 = fine_tuned_prototypes[hd.cosine_similarity(vec_at_400, fine_tuned_prototypes).argmax(dim=1)]

closest_prototype_100, closest_prototype_200, closest_prototype_300

(MAPTensor([[ 1., -1., -1.,  ...,  1.,  1.,  1.]]),
 MAPTensor([[1., 1., 1.,  ..., 1., 1., 1.]]),
 MAPTensor([[1., 1., 1.,  ..., 1., 1., 1.]]))

In [217]:
at_t_100

,video,personID,frameID,AnomalyScore,AnomalyLabel,AnomalyThreshold,RandomSeed,Iteration,VecIndex
30145,6,435,100,0.69669,MAPTensor(False),0.450879,2993,0,30145


In [218]:
at_t_200

,video,personID,frameID,AnomalyScore,AnomalyLabel,AnomalyThreshold,RandomSeed,Iteration,VecIndex
30245,6,435,200,0.506304,MAPTensor(False),0.499652,2993,0,30245


In [219]:
at_t_300

,video,personID,frameID,AnomalyScore,AnomalyLabel,AnomalyThreshold,RandomSeed,Iteration,VecIndex
30345,6,435,300,0.436718,MAPTensor(True),0.499652,2993,0,30345


In [220]:
at_t_400

,video,personID,frameID,AnomalyScore,AnomalyLabel,AnomalyThreshold,RandomSeed,Iteration,VecIndex
30445,6,435,400,0.436718,MAPTensor(True),0.499652,2993,0,30445


In [221]:
interpret_vec(vec_at_100)

Vector: Aspect: middle, Time: normal, Position:5, Posture: standing


In [222]:
interpret_vec(closest_prototype_100)

Vector: Aspect: middle, Time: normal, Position:2, Posture: standing


In [223]:
interpret_vec(vec_at_200)

Vector: Aspect: close, Time: long, Position:6, Posture: standing


In [224]:
interpret_vec(closest_prototype_200)

Vector: Aspect: middle, Time: long, Position:2, Posture: standing


In [225]:
interpret_vec(vec_at_300)

Vector: Aspect: close, Time: long, Position:6, Posture: standing


In [226]:
interpret_vec(closest_prototype_300)

Vector: Aspect: middle, Time: long, Position:2, Posture: standing


In [227]:
interpret_vec(vec_at_400)

Vector: Aspect: close, Time: long, Position:6, Posture: standing


In [228]:
interpret_vec(closest_prototype_400)

Vector: Aspect: middle, Time: long, Position:2, Posture: standing
